In [1]:
import pandas as pd

# Read the Excel files into DataFrames
staff_df = pd.read_excel("staff.xlsx")
types_df = pd.read_excel("types.xlsx")
departments_df = pd.read_excel("departments.xlsx")

# Merge the DataFrames
merged_df = staff_df.merge(types_df, on="Project", how="left")
merged_df = merged_df.merge(departments_df, on="Job", how="left")

# Save the merged DataFrame to an Excel file
merged_df.to_excel("all.xlsx", index=False)

print("Data merged and saved to 'all.xlsx' successfully!")

C:\Users\Omar Essam\AppData\Local\Temp\ipykernel_5220\3885094727.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Data merged and saved to 'all.xlsx' successfully!


In [2]:
# Read the merged data from "all.xlsx"
df = pd.read_excel("all.xlsx")

# Group by project, job title, and type, counting occurrences in each month
grouped_df = df.groupby(["Project", "Job", "Type"])["Month"].value_counts().unstack(fill_value=0)

# Find the peak number for each job title within each project-type combination
peak_counts = grouped_df.max(axis=1)

# Reset the index to include project, job title, and type
result = peak_counts.reset_index()

# Rename the column before saving
result = peak_counts.reset_index().rename(columns={0: "Peak Count"})

# Save the result with the Peak Count column**
result.to_excel("unique_job_titles_peak_counts_with_types.xlsx", index=False)

In [3]:
# Read the data
df = pd.read_excel("unique_job_titles_peak_counts_with_types.xlsx")

# Group by type and job, calculate average peak count
grouped_df = df.groupby(["Type", "Job"])["Peak Count"].mean().round().astype(int).reset_index()

# Create the final DataFrame
final_df = pd.DataFrame()
for type in grouped_df["Type"].unique():
    type_data = grouped_df[grouped_df["Type"] == type]
    final_df = pd.concat([final_df, type_data.set_index("Job").rename_axis(type)], axis=1)  # Removed `.to_frame()`

# Save to Excel with a single sheet
with pd.ExcelWriter("example_projects_structured.xlsx") as writer:
    final_df.to_excel(writer, sheet_name="Example Projects", index=True)

print("Example projects saved to 'example_projects_structured.xlsx' successfully!")

Example projects saved to 'example_projects_structured.xlsx' successfully!


In [5]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# Read the data
df = pd.read_excel("unique_job_titles_peak_counts_with_types.xlsx")

# Get user input for type and duration
user_type = input("Enter the type: ")
user_duration = int(input("Enter the duration in months: "))


In [6]:
def format_month_header(date_str):
  """Converts a date string to the desired format."""
  date = pd.to_datetime(date_str)  # Convert string to date object
  return f"{date.month_name()}-{date.year}"

def generate_bell_curve_counts(duration, peak_count):
    """Generates counts in a bell curve pattern with zeros at the beginning and end."""
    counts = np.zeros(duration)  # Start with zeros
    midpoint = duration // 2

    # Introduce the multiplier to adjust peak scaling
    multiplier = 1.25  # Adjust as needed to reach the desired peak value

    # Fill counts from midpoint to both ends with a bell curve shape
    for i in range(midpoint):
        count = int(peak_count * multiplier * (1 - (abs(midpoint - i) / midpoint) ** 1.5))
        counts[midpoint - i] = count
        counts[midpoint + i] = count

    return counts


In [7]:
# Filter data based on user input
filtered_df = df[df["Type"] == user_type]

# Get unique job titles and average peak counts
unique_peaks = filtered_df.groupby("Job")["Peak Count"].mean().round().astype(int).reset_index()

# Create a DataFrame with unique job titles and rounded average peak counts
result_df = unique_peaks.set_index("Job")

# Create date columns, but don't overwrite result_df
months = pd.date_range(start="2024-01-01", periods=user_duration, freq="MS")
result_df = result_df.reindex(columns=result_df.columns.tolist() + list(months))  # Add date columns to existing result_df


# Generate counts in a bell curve pattern for each job title
for job_title in result_df.index:
    peak_count = result_df.loc[job_title, "Peak Count"]

    # Generate counts based on peak count
    if peak_count == 1:
        counts = np.ones(len(months))  # Set flatline of 1s for peak count 1
    else:
        # Indent this block correctly
        counts = generate_bell_curve_counts(len(months), peak_count)  # Use the new function

    # Assign counts to date columns (this line should be outside the if-else block)
    result_df.loc[job_title, months] = counts

# Save the Excel file without formatting
output_filename = f"{user_type}.xlsx"
result_df.to_excel(output_filename)

print(f"Average peak counts for type '{user_type}' saved to '{output_filename}' successfully!")



Average peak counts for type 'Bridges' saved to 'Bridges.xlsx' successfully!


In [8]:
# Read the Excel file back in
df_with_formatted_months = pd.read_excel(output_filename)

# Redefine date_columns based on the column names in the DataFrame
date_columns = list(df_with_formatted_months.filter(like='2024-').columns)

# Format the month column names
df_with_formatted_months = df_with_formatted_months.rename(columns=lambda x: format_month_header(x) if x in date_columns else x)

# Remove index from the first column
df_with_formatted_months.index.name = None

# Overwrite the existing file with the formatted DataFrame
df_with_formatted_months.to_excel(output_filename, index=False)  # Save without index